东道主进入半决赛/决赛/夺冠概率统计

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
summary = spark.read.format("csv").option("header",True).option("encoding","utf-8").option("sep",",").load("../dataset/WorldCupsSummary.csv")
summary.createTempView("summary_tb")
summary.printSchema()

root
 |-- Year: string (nullable = true)
 |-- HostCountry: string (nullable = true)
 |-- Winner: string (nullable = true)
 |-- Second: string (nullable = true)
 |-- Third: string (nullable = true)
 |-- Fourth: string (nullable = true)
 |-- GoalsScored: string (nullable = true)
 |-- QualifiedTeams: string (nullable = true)
 |-- MatchesPlayed: string (nullable = true)
 |-- Attendance: string (nullable = true)
 |-- HostContinent: string (nullable = true)
 |-- WinnerContinent: string (nullable = true)


In [2]:
hostInTop4_2_1 = spark.sql('''
SELECT ROUND(SUM(IF(HostCountry=Winner,1,0))/COUNT(HostCountry),2) AS host_top1_prob,
ROUND(
(SUM(IF(HostCountry=Winner,1,0))
+SUM(IF(HostCountry=Second,1,0)))
/COUNT(HostCountry)
,2) AS host_top2_prob,
ROUND(
(SUM(IF(HostCountry=Winner,1,0))
+SUM(IF(HostCountry=Second,1,0))
+SUM(IF(HostCountry=Third,1,0))
+SUM(IF(HostCountry=Fourth,1,0)))
/COUNT(HostCountry)
,2) 
AS host_top4_prob FROM summary_tb
''')

In [3]:
hostInTop4_2_1.write \
    .format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/worldcup") \
    .option("dbtable", "hostInTop4_2_1") \
    .option("user", "root") \
    .option("password", "") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .mode("append") \
    .save()
hostInTop4_2_1.show()

+--------------+--------------+--------------+
|host_top1_prob|host_top2_prob|host_top4_prob|
+--------------+--------------+--------------+
|          0.24|          0.33|          0.52|
+--------------+--------------+--------------+
